# First Deep Network

## Uso do MNIST Dataset

Recolher dados e tratá-los é um trabalho longo e tedioso. Os dados do MNIST Dataset já estão tratados e formatados, o que nos permite focar mais na modelagem e treinamento da rede neural.

Essa base de dados é composta por 60 mil exemplos de treinamento de dígitos (0-9) manuscritos e 10 mil exemplos de teste. As imagens possuem 28x28 pixels, um total de **784 pixels**. As imagens já estão tratadas com um filtro de *threshold*.

## Objetivo

A ideia é pegar essas imagens, passar pela rede neural e dizer qual o dígito da imagem.

Cada pixel possui dois valores, 0 ou 1.

## Esquema da Rede Neural

- **input** > weights > hidden layer 1 (função de ativação) > weights > hidden layer 2 (função de ativação) > weights > **output layer** (
É um Feed-forwars NN, o dado é sempre passado para frente.)

- Comparação da **saída** com a **saída esperada** por meio da função de custo (Cross Entropy)

- Função de otimização (AdamOptimizer) -> minimizar o custo

- Backpropagation para modificar as weights




In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## One Hot

O termo vem da eletrônica e significa que quando um componente está *hot* isso significa que ele representa algo.

No caso do MNIST Dataset, como estamos trabalhondo com 10 classes de dados, ou seja, os números de 0 a 9, teremos algo como:

0 = [1,0,0,0,0,0,0,0,0,0]

1 = [0,1,0,0,0,0,0,0,0,0]

2 = [0,0,1,0,0,0,0,0,0,0]

...


In [2]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 100

## Constantes

As constantes **n_nodes** representam a quantidade de nós em cada camada, não necessariamene precisam ser do mesmo tamanho.

**n_classes** representa a quantidade de classes de saída possíveis, no caso, algum dos dígitos entre 0 e 9.

**batch_size** representa a quantidade de elementos do conjunto de dados de treinamento que serão utilizados em cada época, no caso serão 100 imagens por época.

In [3]:
# height x width
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

## Placeholders

Os *placeholders* são variáveis que precisarão ter valores atribuídos quando o modelo da rede neural for executada.

**x** representa os dados de entrada, tem seu valor fixado em uma de **None**, valor indeterminada, por 784, que é a quantidade de pixels de cada imagem.

**y** representa os dados de saída.


In [4]:
def neural_network_model(data):
    hl1 = {'weights': tf.Variable(tf.random_normal([784, n_nodes_hl1])),
           'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hl2 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
           'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hl3 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
           'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    #(input_data * weights) + biases
    
    l1 = tf.add(tf.matmul(data, hl1['weights']), hl1['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hl2['weights']), hl2['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hl3['weights']), hl3['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l2, output_layer['weights']), output_layer['biases'])

    return output

## Modelo

As *weigths* e *biases* das camadas são variáveis com valores aleatórios e de dimensões *dado_de_entrada X dado_de_saida*, logo, as camadas internas possuem dimensões *n_nos_camada_anterior X n_nos_camada_atual*.

Utilizando os métodos otimizados de equações aritiméticas do TensorFlow, são aplicadas as equações de *(input_data * weights) + biases* para cada camada, o resultado é utilziado na função de ativação.

In [ ]:
def train_nn(x, y):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    n_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        #train
        for epoch in range(n_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost],
                                feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
            print('Epoch ', epoch, 'of ', n_epochs, '\nloss: ', epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy: ', accuracy.eval({x: mnist.test.images,
                                           y: mnist.test.labels}))

train_nn(x, y)

## Treinamento

As variáveis de **predição**, **custo** e **otimização** são setadas com métodos do TensorFlow.

Foram definidas 10 épocas de treinamento, os dados são retirados do MNIST Dataset e atribuídos às duas variáveis que terão seus valores atribuídos aos placeholders. A sessão é executada com as funções de otimização, custo e os dados.

A precisão é avaliada utilziando o algoritmo treinado com um conjunto de imagens de teste e comparadas com os resultados esperados.

## Vídeos

### [VÍDEO 1](https://www.youtube.com/watch?v=BhpvH5DuVu8&index=46&list=PLQVvvaa0QuDfKTOs3Keq_kaG2P55YRn5v)
### [VÍDEO 2](https://www.youtube.com/watch?v=PwAGxqrXSCs&list=PLQVvvaa0QuDfKTOs3Keq_kaG2P55YRn5v&index=47)